In [14]:
#Data Manipulation
from numpy import *
from datetime import datetime

from src.blackscholes import BS
from tabulate import tabulate

from watermark import watermark
%load_ext watermark
%watermark -a "Kannan Singaravelu" -u -d -v -m -iv

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Kannan Singaravelu

Last updated: 2023-08-23

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.2.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 13, GenuineIntel
CPU cores   : 8
Architecture: 64bit

numpy: 1.21.5



### Newton Method


In [23]:
def newton_iv(className, spot, strike, rate, dte, callprice=None, putprice=None):
    
    x0 = 1                                      # initial guess
    h = 0.001                                   # step size 
    tolerance = 1e-7                            # 7-digit accuracy is desired
    epsilon = 1e-14                             # do not divide by a number smaller than this, some kind of error / floor
    maxiter = 200                               # maximum number of iterations to execute
    
    # function whose root we are trying to find
    # f(x) = Black Scholes Call price - Market Price - defining the f(x) here
    if callprice:
        f = lambda x: eval(className)(spot, strike, rate, dte, x).callPrice - callprice
    if putprice:
        f = lambda x: eval(className)(spot, strike, rate, dte, x).putPrice - putprice
        
    for i in range(maxiter):
        y = f(x0)                               # starting with initial guess
        yprime = (f(x0+h) - f(x0-h))/(2*h)      # central difference, the derivative of the function
        
        if abs(yprime)<epsilon:                 # stop if the denominator is too small
            break                              
        x1 = x0 - y/yprime                      # perform Newton's computation
        
        if (abs(x1-x0) <= tolerance*abs(x1)):   # stop when the result is within the desired tolerance
            break
        x0=x1                                   # update x0 to start the process again
        
    return x1                                   #x1 is a solution within tolerance and maximum numb

In [24]:
#newton iv
newton_iv('BS',100,100,0.02,1,callprice=8)

0.17657213831399154

In [27]:
opt = BS(100,100,0.02,1,0.2)
opt.callPrice

8.916037278572539

In [30]:
newton_iv('BS',100,100,0.02,1,callprice=opt.callPrice)

0.20000000000000015

### Bisection Method

In [31]:
#Bisection Method
def bisection_iv(className,spot,strike,rate,dte,callprice=None,putprice=None,high=500.0,low=0.0):
    
    #this is market price
    if callprice:
        price = callprice
    if putprice and not callprice:
        price = putprice
        
    tolerance = 1e-7
    
    for i in range(1000):
        mid = (high + low) / 2                   # c = (a+b)/2
        if mid <tolerance:
            mid = tolerance
            
        if callprice:
            estimate = eval(className)(spot,strike,rate,dte,mid).callPrice  #Balck Scholes price
        if putprice:
            estimate = eval(className)(spot,strike,rate,dte,mid).putPrice
            
        if round(estimate,6) == price:
            break
        elif estimate > price:
            high = mid              #replace c with b|b = c
        elif estimate < price:
            low = mid               #replace c with a|a = c
            
    return mid

In [33]:
#bisection iv
bisection_iv('BS',100,100,0.02,1,callprice=8.)

0.17657213902566582

In [36]:
bisection_iv('BS',100,100,0.02,1,callprice=8.916037278572539)

0.20000000000000007

### BS Implied Volatility

In [37]:
from vol import BS

ModuleNotFoundError: No module named 'vol'

In [38]:
# Initialize option
option = BS(100,100,0.05,1,0.2, callprice=8)

header = ['Option Price', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', 'IV']
table = [[option.callPrice, option.callDelta, option.gamma, option.callTheta, option.vega, option.callRho, option.impvol]]

print(tabulate(table,header))

TypeError: __init__() got an unexpected keyword argument 'callprice'